In [1]:
import os
import shutil

import cv2
import numpy as np
from joblib import Parallel, delayed

In [2]:
def mask_file_to_yolo(mask_path: str, yolo_label_path: str):
    # Load or generate your segmentation mask
    # Assume mask is a 512x512 NumPy array where each pixel is an integer label
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    object_ids = np.unique(mask[mask > 0])
    
    with open(yolo_label_path, 'w') as f:
        # Find contours for the object with all labels
        for object_id in object_ids:
            label = object_id
            contours, _ = cv2.findContours((mask == label).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Generate the line for each contour found (in case there are multiple instances of the label)
            lines = []
            for contour in contours:
                # Flatten contour points and scale them back to [0, 1] relative coordinates
                contour_points = contour.reshape(-1, 2)
                contour_points = contour_points / 512  # Scale points to (0-1) range if required by YOLO
                points = ' '.join(f'{x:.6f} {y:.6f}' for x, y in contour_points)
                
                # Format: <class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
                line = f"{label - 1} {points}"
                lines.append(line)
        
            # Print result
            for line in lines:
                f.write(line + '\n')
                

In [4]:
for dataset in ['train', 'val']:
    image_path = f'./data/Public_leaderboard_data/{dataset}_images_clean'
    yolo_image_path = f'./data/leaderboard_yolo/images/{dataset}'
    os.makedirs(yolo_image_path, exist_ok=True)
    label_path = f'./data/Public_leaderboard_data/{dataset}_labels'
    yolo_label_path = f'./data/leaderboard_yolo/labels/{dataset}'
    os.makedirs(yolo_label_path, exist_ok=True)
    
    print('Processing', dataset, 'set.')
    
    def parallel_convert(folder):
        for img in os.listdir(os.path.join(label_path, folder)):
            mask_file_to_yolo(os.path.join(label_path, folder, img), os.path.join(yolo_label_path, f'{folder}_{img.replace(".png", ".txt")}'))
            shutil.copy(os.path.join(image_path, folder, img), os.path.join(yolo_image_path, f'{folder}_{img}'))
            
            
    Parallel(n_jobs=-1)(delayed(parallel_convert)(folder) for folder in os.listdir(label_path))

Processing train set.
Processing val set.
